## Zad. 1

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve

In [2]:
# wczytanie danych
san = pd.read_csv('dane/san.csv', index_col=0, skiprows=2)
san.index = pd.to_datetime(san.index, format='%Y-%m-%d')
san = san.iloc[:, 0]

erste_group = pd.read_csv('dane/EBS_VI.csv', index_col=0, skiprows=2)
erste_group.index = pd.to_datetime(erste_group.index, format='%Y-%m-%d')
erste_group = erste_group.iloc[:, 0].to_frame(name='Price')

In [3]:
# ostatnia znana cena akcji
round(san.iloc[-1], 3)

6.85

In [4]:
# premia do opcji sprzedaży
S = 6.85
K = S
T = 0.25
sigma = san.pct_change().dropna().std() * np.sqrt(252)
R = 0.03
n = 10**4  # liczba akcji
fixed_forward_price = 6.90

In [5]:
# cena Blacka-Scholesa na opcję sprzedaży
def black_scholes(s=S, k=K, t=T, sig=sigma, r=R):
    d1 = (-np.log(s/k) - (r - sig**2/2) * t) / (sig*np.sqrt(t))
    d2 = (-np.log(s/k) - (r + sig**2/2) * t) / (sig*np.sqrt(t))
    return k * np.exp(-r * t) * norm.cdf(d1) - s * norm.cdf(d2)


round(black_scholes(), 2)

0.52

In [6]:
premium = n * round(black_scholes(), 2)
premium

5200.0

Santander Polska posiada 10 000 akcji Banco Santander S.A. (wartość: 68 500 USD). W celu zabezpieczenia przed ryzykiem spadku cen akcji, możliwe są trzy strategie:
1. Podjęcie ryzyka.
2. Zakup kontraktu forward na sprzedaż akcji z terminem wykonania 3 miesiące. Cena wykonania kontaktu wynosi 6.90 USD/akcja.
3. Zakup opcji sprzedaży na instrument podstawowy z ceną wykonania 6.85 USD/akcja, terminem wykonania za 3 miesiące i ceną wykonania (premią) 0.52 USD/akcja.

In [7]:
def option3(f_vals, strike, premium):
    res = []
    executed = []
    for val in f_vals:
        payoff = max(strike - val, 0)
        net = (val + payoff) * n - premium 
        res.append(net)
        executed.append("T" if payoff > 0 else "F")
    return res, executed

In [8]:
future_vals = np.array([6.6, 6.7, 6.8, 6.85, 6.9, 7., 7.1])
result = pd.DataFrame({
'Wartość kontraktu': future_vals * n,
'Efekt strategii 1 – podjęcie ryzyka': future_vals * n,
'Efekt strategii 2 - sprzedaż kontraktu forward': [6.9 * n] * 7,
'Efekt strategii 3 - zakup opcji sprzedaży': option3(future_vals, K, premium)[0],
'Wykonanie opcji sprzedaży': option3(future_vals, K, premium)[1]},
    index = future_vals)
result

,Wartość kontraktu,Efekt strategii 1 – podjęcie ryzyka,Efekt strategii 2 - sprzedaż kontraktu forward,Efekt strategii 3 - zakup opcji sprzedaży,Wykonanie opcji sprzedaży
6.60,66000.0,66000.0,69000.0,63300.0,T
6.70,67000.0,67000.0,69000.0,63300.0,T
6.80,68000.0,68000.0,69000.0,63300.0,T
6.85,68500.0,68500.0,69000.0,63300.0,F
6.90,69000.0,69000.0,69000.0,63800.0,F
7.00,70000.0,70000.0,69000.0,64800.0,F
7.10,71000.0,71000.0,69000.0,65800.0,F


In [9]:
sigma_daily = sigma / np.sqrt(252/60)

np.random.seed(42)
sim_log_returns = np.random.normal(loc=0, scale=sigma_daily, size=n)
sim_prices = S * np.exp(sim_log_returns)

# Strategia 1 
strategy1_value = sim_prices * n
losses1 = (S * n) - strategy1_value

# Strategia 2
strategy2_value = np.full(n, fixed_forward_price * n)
losses2 = np.zeros(n)

# Strategia 3
payoffs = np.maximum(K - sim_prices, 0) 
strategy3_value = (sim_prices + payoffs) * n - premium
losses3 = (S * n) - strategy3_value

VaR_95_1 = np.quantile(losses1, 0.95)
VaR_95_2 = np.quantile(losses2, 0.95)
VaR_95_3 = np.quantile(losses3, 0.95)

In [10]:
pd.DataFrame({'3-miesięczny VaR(95%)': [VaR_95_1 , VaR_95_2, VaR_95_3]})

,3-miesięczny VaR(95%)
0,18991.980911
1,0.000000
2,5200.000000


# Zad 2
$$
\text{Default Probability} = \text{EDF} = \Phi(-d_2)
$$

1-year default probability based on the assumption that asset values follow a lognormal distribution.

### Banco Santander, S.A. (SAN)

In [11]:
san = san.to_frame(name='Price')
# --- Constants ---
shares_outstanding = 14.87e9  # shares
debt = 372.97e9               # total debt in USD
r = 0.04407                   # risk-free rate U.S. 10 Year Treasury
K = 0.031                     # dividend yield
T = 1.0                       # time horizon (1 year)
t = 0                         # t in [0, T]

# --- Compute market equity value and volatility ---
san['log_return'] = np.log(san['Price'] / san['Price'].shift(1))
equity_vol = san['log_return'].std() * np.sqrt(252)  # annualized
last_price = san['Price'].iloc[-1]
equity_value = last_price * shares_outstanding  # market cap

# --- KMV Model Equations ---
def equations(p):
    V, sigma_V = p
    d1 = (np.log(V / debt) + ((r - K) + 0.5 * sigma_V ** 2) * (T - t)) / (sigma_V * np.sqrt(T - t))
    d2 = d1 - sigma_V * np.sqrt(T - t)
    eq1 = V * norm.cdf(d1) - debt * np.exp(-r * (T - t)) * norm.cdf(d2) - equity_value
    eq2 = V * norm.cdf(d1) * sigma_V - equity_value * equity_vol
    return (eq1, eq2)

# --- Initial guesses ---
V0 = equity_value + debt
sigma_V0 = equity_vol
V, sigma_V = fsolve(equations, (V0, sigma_V0))

# --- Calculate Distance to Default ---
d1 = (np.log(V / debt) + (r + 0.5 * sigma_V ** 2) * T) / (sigma_V * np.sqrt(T))
d2 = d1 - sigma_V * np.sqrt(T)
DD = d2
EDF = norm.cdf(-DD)

# --- Output ---
print(f"Market Equity Value: ${equity_value:,.2f}")
print(f"Equity Volatility: {equity_vol:.4f}")
print(f"Estimated Asset Value: ${V:,.2f}")
print(f"Estimated Asset Volatility: {sigma_V:.4f}")
print(f"Distance to Default (DD): {DD:.4f}")
print()
print(f"Expected Default Frequency (EDF): {EDF:.4%}")

Market Equity Value: $101,859,498,581.89
Equity Volatility: 0.4044
Estimated Asset Value: $458,753,521,030.55
Estimated Asset Volatility: 0.0904
Distance to Default (DD): 2.7325

Expected Default Frequency (EDF): 0.3143%


### Erste Group Bank AG (EBS.VI)

In [12]:
# --- Constants ---
shares_outstanding = 388.13e6  # shares
debt = 65.63e9                 # total debt in EUR
r = 0.02535                    # risk-free rate EU10Y-EU:Eurobonds
K = 0.0423                     # dividend yield
T = 1.0                        # time horizon (1 year)
t = 0                          # t in [0, T]

# --- Compute market equity value and volatility ---
erste_group['log_return'] = np.log(erste_group['Price'] / erste_group['Price'].shift(1))
equity_vol = erste_group['log_return'].std() * np.sqrt(252)  # annualized
last_price = erste_group['Price'].iloc[-1]
equity_value = last_price * shares_outstanding  # market cap

# --- KMV Model Equations ---
def equations(p):
    V, sigma_V = p
    d1 = (np.log(V / debt) + ((r - K) + 0.5 * sigma_V ** 2) * (T - t)) / (sigma_V * np.sqrt(T - t))
    d2 = d1 - sigma_V * np.sqrt(T - t)
    eq1 = V * norm.cdf(d1) - debt * np.exp(-r * (T - t)) * norm.cdf(d2) - equity_value
    eq2 = V * norm.cdf(d1) * sigma_V - equity_value * equity_vol
    return (eq1, eq2)

# --- Initial guesses ---
V0 = equity_value + debt
sigma_V0 = equity_vol
V, sigma_V = fsolve(equations, (V0, sigma_V0))

# --- Calculate Distance to Default ---
d1 = (np.log(V / debt) + (r + 0.5 * sigma_V ** 2) * T) / (sigma_V * np.sqrt(T))
d2 = d1 - sigma_V * np.sqrt(T)
DD = d2
EDF = norm.cdf(-DD)

# --- Output ---
print(f"Market Equity Value: ${equity_value:,.2f}")
print(f"Equity Volatility: {equity_vol:.4f}")
print(f"Estimated Asset Value: ${V:,.2f}")
print(f"Estimated Asset Volatility: {sigma_V:.4f}")
print(f"Distance to Default (DD): {DD:.4f}")
print()
print(f"Expected Default Frequency (EDF): {EDF:.4%}")

Market Equity Value: $24,105,148,087.58
Equity Volatility: 0.3683
Estimated Asset Value: $88,095,002,169.27
Estimated Asset Volatility: 0.1010
Distance to Default (DD): 3.1142

Expected Default Frequency (EDF): 0.0922%
